In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# ==================== 配置参数 ====================
CONFIG = {
    'num_samples': 500,
    'image_size': (64, 64),
    'base_filters': 32,
    'depth': 3,
    'epochs': 60,
    'batch_size': 32,
    'learning_rate': 0.001,
}

# ==================== 数据生成 ====================
def create_circle_mask(size=64, center_shift=0):
    """生成含有圆形的矩阵"""
    y, x = np.ogrid[-1:1:size*1j, -1:1:size*1j]
    radius = 0.5
    circle = (x**2 + y**2) <= radius**2
    
    # 圆形内部数值大（1.0），外部数值小（0.2）
    mask = np.where(circle, 1.0, 0.2)
    return mask

def generate_synthetic_data(num_samples=500, image_size=(64, 64)):
    """生成圆形数据集"""
    X = np.array([create_circle_mask(image_size[0]) for _ in range(num_samples)])
    y = np.array([create_circle_mask(image_size[0]) for _ in range(num_samples)])
    
    # 数据增强：旋转、加噪声
    X = np.array([np.rot90(x, np.random.randint(0, 4)) for x in X])
    y = np.array([np.rot90(y_, np.random.randint(0, 4)) for y_ in y])
    X += np.random.normal(0, 0.02, X.shape)
    X = np.clip(X, 0, 1)
    
    # 归一化
    X = (X - X.min()) / (X.max() - X.min() + 1e-8)
    y = (y - y.min()) / (y.max() - y.min() + 1e-8)
    return X, y

def postprocess_prediction(pred, threshold=0.8):
    """按范围强化预测结果：>0.8向上取整，<0.3向下取整"""
    result = pred.copy()
    result[result >= threshold] = 1.0
    result[result <= 0.3] = 0.0
    return result

# ==================== 模型构建 ====================
def build_unet(input_shape=(64, 64, 1), base_filters=32, depth=3):
    """轻量级U-Net"""
    inputs = layers.Input(shape=input_shape)
    x = inputs
    skips = []
    
    # 编码器
    for i in range(depth):
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        skips.append(x)
        x = layers.MaxPooling2D((2, 2))(x)
    
    # Bottleneck
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    
    # 解码器
    for i in reversed(range(depth)):
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, skips[i]])
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
    
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate']),
                  loss='binary_crossentropy', metrics=['mse', 'mae'])
    return model

# ==================== 主程序 ====================
# 生成数据
print("生成圆形数据集...")
X, y = generate_synthetic_data(CONFIG['num_samples'], CONFIG['image_size'])
X = X[..., np.newaxis]
y = y[..., np.newaxis]

# 创建结果目录
result_dir = os.path.join('results')
os.makedirs(result_dir, exist_ok=True)

# 训练模型
print("训练轻量级U-Net模型...")
model = build_unet(input_shape=(64, 64, 1), base_filters=CONFIG['base_filters'], depth=CONFIG['depth'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X, y, epochs=CONFIG['epochs'], batch_size=CONFIG['batch_size'],
                   validation_split=0.2, callbacks=[early_stopping], verbose=1)

val_mse = history.history['val_mse'][-1]
val_mae = history.history['val_mae'][-1]
print(f"最终验证集MSE: {val_mse:.4f}, MAE: {val_mae:.4f}")

# 保存损失曲线
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('U-Net训练曲线')
plt.savefig(os.path.join(result_dir, "training_loss.png"), dpi=100)
plt.close()

# 预测并可视化
idx = np.random.randint(0, X.shape[0])
sample_input = X[idx:idx+1]
sample_true = y[idx]
sample_pred = model.predict(sample_input)[0]
sample_pred_enhanced = postprocess_prediction(sample_pred, threshold=0.8)

plt.figure(figsize=(16, 4))
plt.subplot(1, 4, 1)
plt.title('Input (Circle)')
plt.imshow(sample_input[0, :, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 2)
plt.title('True Label')
plt.imshow(sample_true[:, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 3)
plt.title('Raw Prediction')
plt.imshow(sample_pred[:, :, 0], cmap='gray')
plt.colorbar()

plt.subplot(1, 4, 4)
plt.title('Enhanced Prediction')
plt.imshow(sample_pred_enhanced[:, :, 0], cmap='gray')
plt.colorbar()

plt.tight_layout()
plt.savefig(os.path.join(result_dir, "prediction_comparison.png"), dpi=100)
plt.close()

# 保存指标
with open(os.path.join(result_dir, "metrics.txt"), "w") as f:
    f.write(f"Circle Detection U-Net\n")
    f.write(f"Val MSE: {val_mse:.4f}\n")
    f.write(f"Val MAE: {val_mae:.4f}\n")
    f.write(f"Samples: {CONFIG['num_samples']}, Epochs: {len(history.history['loss'])}\n")

print("训练完成！结果已保存到results目录。")

# U4EIP：基于 U-Net 的电磁特性恢复系统

## 📋 概述

一个基于卷积神经网络的电磁特性恢复系统，通过轻量级U-Net模型从电磁波传播数据中精确恢复介质的电磁特性（如介电常数分布）。

---

## 🎯 设计原理

### 1. **问题定义**
- **输入**：模拟的电磁波传播数据（64×64矩阵）
- **输出**：介质的电磁特性分布（64×64矩阵）
- **任务**：从观测数据反演介质物理参数

### 2. **核心思路**

#### 数据层：圆形图案生成
```
圆形掩膜 (Circle Mask)
├─ 圆心内部: 数值 = 1.0 (深色，代表高介电常数)
├─ 圆心外部: 数值 = 0.2 (浅色，代表低介电常数)
└─ 模拟真实电磁介质的空间分布特征
```

数据增强策略：
- **旋转变换**：增加模型对方向不变性的学习
- **高斯噪声**：模拟真实测量误差（σ=0.02）
- **归一化处理**：保证数值稳定性和收敛速度

#### 模型层：轻量级U-Net架构
```
编码器 (Encoder)
    ↓ Conv2D × 2 → MaxPooling (× 3 层)
    ↓
Bottleneck
    ↓ Conv2D × 2
    ↓
解码器 (Decoder)
    ↓ UpSampling → Concat → Conv2D × 2 (× 3 层)
    ↓
输出层 (Sigmoid激活)
```

#### 训练层：二值分类损失
```python
损失函数: Binary Crossentropy
优化器: Adam (lr=0.001)
正则化: 早停机制 (patience=10)
```

#### 推理层：阈值强化处理
```python
后处理规则:
├─ 高值区域 (≥0.8) → 向上取整 = 1.0
├─ 低值区域 (≤0.3) → 向下取整 = 0.0
└─ 中间值 (0.3~0.8) → 保持原值
```

**强化效果**：
- 去除预测中的模糊不确定性
- 增强二值化分割效果
- 接近真实物理模型（介质存在/不存在）

---


### 3. **可配置化架构**
```python
CONFIG = {
    'num_samples': 500,      # 易于调整数据规模
    'base_filters': 32,      # 可扩展模型容量
    'depth': 3,              # 灵活调整网络深度
    'epochs': 60,            # 自适应训练轮数
    'learning_rate': 0.001,  # 独立的学习率控制
}
```

### 4. **模块化代码结构**
```
数据生成模块 ──→ 模型构建模块 ──→ 训练评估模块 ──→ 后处理与可视化
  (独立函数)      (独立函数)       (主程序逻辑)      (完整的结果输出)
```

**可维护性优势**：
- 各模块独立，易于单元测试
- 配置集中管理，便于参数调优
- 函数接口清晰，扩展性强

---


## 🔄 工作流程

```
1. 数据生成
   ├─ 生成500个圆形样本
   ├─ 旋转与噪声增强
   └─ 归一化处理

2. 模型训练
   ├─ 构建轻量级U-Net
   ├─ 60个epochs（可能早停）
   └─ 二值交叉熵优化

3. 性能评估
   ├─ 计算MSE & MAE
   ├─ 绘制训练曲线
   └─ 保存指标文件

4. 预测与强化
   ├─ 模型推理（原始预测）
   ├─ 阈值强化处理
   └─ 生成对比可视化

5. 结果输出
   ├─ training_loss.png (训练曲线)
   ├─ prediction_comparison.png (4图对比)
   └─ metrics.txt (性能指标)
```

---

## 🚀 运行指南

### 环境配置
```bash
pip install tensorflow numpy matplotlib
```

### 执行训练
```bash
cd src
python -c "exec(open('main.ipynb').read())"
# 或在Jupyter中运行main.ipynb
```

### 查看结果
```bash
ls -la results/
# 输出：
# - training_loss.png
# - prediction_comparison.png
# - metrics.txt
```

---

## 📈 预期结果

### 验证集指标
- **MSE**: < 0.01（目标）
- **MAE**: < 0.05（目标）

### 可视化对比
四张并排图像展示：
1. **Input (Circle)** - 输入电磁波数据
2. **True Label** - 真实介质分布
3. **Raw Prediction** - 模型原始预测
4. **Enhanced Prediction** - 阈值强化后结果

---

## 🔬 物理意义

| 矩阵数值 | 物理含义 | 材料例子 |
|---------|--------|---------|
| 0.2 | 低介电常数 | 空气、真空 |
| 1.0 | 高介电常数 | 陶瓷、玻璃 |

**应用场景**：
- 地下水资源勘探（GPR雷达）
- 医学成像（MRI信号反演）
- 工业无损检测（缺陷识别）
